### Game Environment

- each game, has many rounds, which has many periods, which has fixed bid-ask and buy-sell time steps. 
- token distributions are fixed within a round, but values vary accross periods
- at start of a game, the following information is revealed: 
    - Game start: (public information) num_rounds, num_periods, time_steps, num_tokens, distribution
    - Round start: (private information) token values are revealed to each trader.
    - Each seller has ntokens number of tokens, buyers have none. 
- at start of round, we can compute the demand and supply curves from the token_values and the competitive equilibrium. At this price, we would see the market instantly clear. 


In [1]:
import numpy as np
from numba import jit

def game_setup(max_rounds, max_periods, max_tokens, max_buyers, max_sellers, max_K, minprice = 1, maxprice = 2000, nsteps = 25):
    nrounds = np.random.randint(1,max_rounds)
    nperiods = np.random.randint(1,max_periods)
    nbuyers = np.random.randint(1,max_buyers)
    nsellers = np.random.randint(1,max_sellers)
    ntokens = np.random.randint(1,max_tokens)
    gametype_1 = np.random.randint(1,max_K)
    gametype_2 = np.random.randint(1,max_K)
    gametype_3 = np.random.randint(1,max_K)
    gametype_4 = np.random.randint(1,max_K)
    R1 = 3 ** gametype_1 - 1
    R2 = 3 ** gametype_2 - 1
    R3 = 3 ** gametype_3 - 1
    R4 = 3 ** gametype_4 - 1
    return nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4, minprice, maxprice, nsteps

@jit
def round_setup(ntokens, nbuyers, nsellers, R1, R2, R3, R4):
    A = np.random.uniform(0,R1)
    B = np.random.uniform(0,R2)
    C = np.random.uniform(0,R3,(nbuyers + nsellers,1))
    D = np.random.uniform(0,R4,(nbuyers + nsellers,ntokens))
    token_values = A + B + C + D
    return token_values

In [2]:
# Game parameters 
nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4, minprice, maxprice, nsteps = game_setup(7,7,7,7,7,7)
token_values = round_setup(ntokens, nbuyers, nsellers, R1, R2, R3, R4)
print(nrounds, nperiods, ntokens, nbuyers, nsellers)
print(token_values) # player (row), token ID (col)

2 3 6 2 3
[[736.54181048 149.35572663 470.55511145 112.35547819 577.51729169
  727.15883714]
 [628.85971386 141.16075672 764.77642182 545.91929148 379.6760575
  467.69707801]
 [405.2637278  349.43450021 657.52066046 219.04374037 673.93095554
  285.768845  ]
 [722.51684337 692.71031671 794.81668376 504.84646005 757.40953082
  113.81898153]
 [450.58429948  98.66063529 630.16692874 630.5478808  774.96916978
  490.48803488]]


### Competitive Equilibrium

In [3]:
@jit
def token_value_breakup(token_values,nbuyers,nsellers):
    buyer_token_values = token_values[0:nbuyers,:]
    max_eqbm_price = np.max(buyer_token_values)
    seller_token_values = token_values[nbuyers:nbuyers+nsellers,:]
    min_eqbm_price = np.min(seller_token_values)
    return buyer_token_values, seller_token_values, max_eqbm_price, min_eqbm_price

@jit
def compute_demand_curve(buyer_token_values,nbuyers,ntokens,min_eqbm_price,max_eqbm_price,granularity=1000):
    P_grid = np.linspace(min_eqbm_price,max_eqbm_price,granularity)
    demand_schedule = np.zeros((granularity),dtype = 'int')
    for i, p in enumerate(P_grid):
        demand = np.sum(p<buyer_token_values) # how many tokens are buyers ready to buy at this price
        demand_schedule[i] = demand
    return demand_schedule, P_grid

@jit
def compute_supply_curve(seller_token_values,nsellers,ntokens,min_eqbm_price,max_eqbm_price,granularity=1000):
    P_grid = np.linspace(min_eqbm_price,max_eqbm_price,granularity)
    supply_schedule = np.zeros((granularity), dtype = 'int')
    for i, p in enumerate(P_grid):
        supply = np.sum(p>seller_token_values) # how much sellers are ready to sell at this price
        supply_schedule[i] = supply
    return supply_schedule, P_grid

@jit
def find_equilibrium(demand_schedule,supply_schedule,P_grid):
    p_eqbm = []
    q_eqbm = None
    for i, p in enumerate(P_grid):
        if demand_schedule[i] == supply_schedule[i]: # when sellers are ready to sell
            p_eqbm.append(p)
            q_eqbm = demand_schedule[i]
    return p_eqbm, q_eqbm

def compute_demand_supply(token_values, plot = False):
    buyer_token_values, seller_token_values, max_eqbm_price, min_eqbm_price = token_value_breakup(token_values,nbuyers,nsellers)
    demand_schedule, P_grid = compute_demand_curve(buyer_token_values,nbuyers,ntokens,min_eqbm_price,max_eqbm_price)
    supply_schedule, P_grid = compute_supply_curve(seller_token_values,nsellers,ntokens,min_eqbm_price,max_eqbm_price)
    p_eqbm, q_eqbm = find_equilibrium(demand_schedule,supply_schedule,P_grid)
    
    if plot == True:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(8, 8))
        aspect_ratio = 1.5
        plt.style.use('ggplot')
        plt.plot(demand_schedule, P_grid, label = 'Demand Curve', c = 'black')
        plt.plot(supply_schedule, P_grid, label = 'Supply Curve', c = 'black')
        #plt.scatter([q_eqbm]*len(p_eqbm),p_eqbm, label = 'Equilibrium', s = 10)
        plt.axhline(y=np.nanmean(p_eqbm), color='blue', linestyle='--', label='Mean Eqbm Price', c = 'black')

        try:
            print('\nEqbm price range:', np.round(np.min(p_eqbm),0),'to',np.round(np.max(p_eqbm),0))
        except:
            print('\nEqbm price range:', p_eqbm)
        print('Eqbm quantity:', q_eqbm)
        plt.legend()
        plt.show()
    return p_eqbm, q_eqbm

def plot_period_results(period_bids, period_asks, period_prices, period_sales, token_values, ntokens, nbuyers, nsellers, R1, R2, R3, R4):
    buyer_token_values, seller_token_values, max_eqbm_price, min_eqbm_price = token_value_breakup(token_values,nbuyers,nsellers)
    demand_schedule, P_grid = compute_demand_curve(buyer_token_values,nbuyers,ntokens,min_eqbm_price,max_eqbm_price)
    supply_schedule, P_grid = compute_supply_curve(seller_token_values,nsellers,ntokens,min_eqbm_price,max_eqbm_price)
    p_eqbm, q_eqbm = find_equilibrium(demand_schedule,supply_schedule,P_grid)

    import matplotlib.pyplot as plt
    plt.figure(figsize=(5,5))
    aspect_ratio = 1.5
    plt.style.use('ggplot')
    plt.plot(demand_schedule, P_grid, label = 'Demand Curve')
    plt.plot(supply_schedule, P_grid, label = 'Supply Curve')
    count = 0
    prices = []

    for i in range(nsteps):
        bids = period_bids[i]
        asks = period_asks[i]
        price = period_prices[i]
        sales = period_sales[i]
        if price != np.nan:
            plt.scatter(bids*0+sales, bids, s = 10, marker = 'x', c = 'red')
            plt.scatter(asks*0+sales, asks, s = 10, marker = 'o', c = 'blue')
            plt.scatter(price*0+sales, price, s = 20, marker = '^', c = 'green')
        else:
            pass

    plt.axhline(y=np.nanmean(period_prices), color='green', linestyle='--', label='Mean Real Prices')
    plt.axhline(y=np.nanmean(p_eqbm), color='black', linestyle='--', label='Mean Eqbm Prices')
    print('Eqbm quantity:', q_eqbm)
    plt.legend()
    plt.show()
    
def profit_analysis(period_profits_buyers, period_profits_sellers):
    import pandas as pd
    col_names = ['round', 'period', 'step'] + ['B'+str(i) for i in range(nbuyers)]
    df_buyer_profits = pd.DataFrame(period_profits_buyers, columns = col_names)
    col_names = ['round', 'period', 'step'] + ['S'+str(i) for i in range(nsellers)]
    df_seller_profits = pd.DataFrame(period_profits_sellers, columns = col_names)
    print('Avg Buyer Profits:', df_buyer_profits.mean()[3:].to_numpy())
    print('Avg Seller Profits:', df_seller_profits.mean()[3:].to_numpy())


### Trading Strategies

- Truthteller: just bids/asks at the true value
- Sniper (Kaplan): waits for bid-ask to close and then jumps in and steal the deal
- Creeper (EL): bids aggresively and then lowers margin based on others'
- Random (ZI): bids randomly while respecting budget
- Forecaster (GD): predicts current bid/ask that would be accepted based on historical data

In [4]:
class TradingStrategy:
    def __init__(self, token_values, buyer=True):
        self.buyer = buyer
        self.period_profit = 0
        self.round_profit = 0
        self.game_profit = 0
        self.token_values = list(np.round(np.sort(token_values, kind='quicksort')[::-1],1))
        if self.buyer == True:
            self.num_tokens_held = 0
            self.tokens_held = []
        else:
            self.num_tokens_held = len(self.token_values)
            self.tokens_held = self.token_values

    def transact(self, price):
        if self.buyer == True:
            self.num_tokens_held += 1
            self.tokens_held.append(self.value)
            self.period_profit += self.value-price
        else:
            self.num_tokens_held = self.num_tokens_held -1
            self.tokens_held.remove(self.value)
            self.period_profit += price-self.value
            
    def reset(self):
        if self.buyer == True:
            self.num_tokens_held = 0
            self.tokens_held = []
            self.period_profit = 0
        else:
            self.num_tokens_held = len(self.token_values)
            self.tokens_held = self.token_values
            self.period_profit = 0

    def describe(self):
        print(f"\n")
        print(f"Buyer: {self.buyer}")
        print(f"Tokens Held: {self.tokens_held}")
        print(f"Token Values: {self.token_values}")
        print(f"Period Profit: {self.period_profit}")
        print(f"Round Profit: {self.round_profit}")
        print(f"Game Profit: {self.game_profit}")

buyer = TradingStrategy(token_values[0], buyer = True)
buyer.describe()
seller = TradingStrategy(token_values[1], buyer = False)
seller.describe()



Buyer: True
Tokens Held: []
Token Values: [736.5, 727.2, 577.5, 470.6, 149.4, 112.4]
Period Profit: 0
Round Profit: 0
Game Profit: 0


Buyer: False
Tokens Held: [764.8, 628.9, 545.9, 467.7, 379.7, 141.2]
Token Values: [764.8, 628.9, 545.9, 467.7, 379.7, 141.2]
Period Profit: 0
Round Profit: 0
Game Profit: 0


In [5]:
# Truthteller
class TruthTeller(TradingStrategy):
    def __init__(self, token_values, buyer=True):
        super().__init__(token_values, buyer)

    def bid(self):
        if self.num_tokens_held == len(self.token_values):
            return np.nan
        n = self.num_tokens_held + 1 # index of token
        self.value = np.partition(self.token_values, -n)[-n] # get nth max
        self.bid_amount = self.value
        return self.bid_amount
    
    def ask(self):
        if self.num_tokens_held == 0:
            return np.nan
        n = self.num_tokens_held
        self.value = np.partition(self.token_values, -n)[-n]
        self.ask_amount = self.value
        return self.ask_amount
    
    def buy(self, current_bid, current_ask):
        if current_ask <= current_bid:
            return True
        else:
            return False
        
    def sell(self, current_bid, current_ask):
        if current_ask <= current_bid:
            return True
        else:
            return False

In [6]:
buyer = TruthTeller(token_values[0],buyer=True)
seller = TruthTeller(token_values[1],buyer=False)
print(buyer.tokens_held, buyer.num_tokens_held, buyer.token_values, buyer.period_profit)
print(seller.tokens_held, seller.num_tokens_held, seller.token_values, seller.period_profit)
for i in range(ntokens+1):
    print('\n')
    print(buyer.bid(), seller.ask())
    if buyer.buy(buyer.bid_amount, seller.ask_amount) == False or seller.sell(buyer.bid_amount, seller.ask_amount) == False:
        break
    price = np.random.choice([buyer.bid_amount, seller.ask_amount])
    print(price)
    buyer.transact(price)
    seller.transact(price)
    print(buyer.tokens_held, buyer.num_tokens_held, buyer.token_values, buyer.period_profit)
    print(seller.tokens_held, seller.num_tokens_held, seller.token_values, seller.period_profit)

[] 0 [736.5, 727.2, 577.5, 470.6, 149.4, 112.4] 0
[764.8, 628.9, 545.9, 467.7, 379.7, 141.2] 6 [764.8, 628.9, 545.9, 467.7, 379.7, 141.2] 0


736.5 141.2
736.5
[736.5] 1 [736.5, 727.2, 577.5, 470.6, 149.4, 112.4] 0.0
[764.8, 628.9, 545.9, 467.7, 379.7] 5 [764.8, 628.9, 545.9, 467.7, 379.7] 595.3


727.2 379.7
727.2
[736.5, 727.2] 2 [736.5, 727.2, 577.5, 470.6, 149.4, 112.4] 0.0
[764.8, 628.9, 545.9, 467.7] 4 [764.8, 628.9, 545.9, 467.7] 942.8


577.5 467.7
467.7
[736.5, 727.2, 577.5] 3 [736.5, 727.2, 577.5, 470.6, 149.4, 112.4] 109.80000000000001
[764.8, 628.9, 545.9] 3 [764.8, 628.9, 545.9] 942.8


470.6 545.9


### Game Simulation

In [169]:
# initialize game parameters 
verbose = 0
#nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4, minprice, maxprice, nsteps = game_setup(7,7,7,7,7,7)
nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4 = 4,4,4,4,4, 20, 20, 20, 20
print(f"--------Trading Game Start--------")
print(f"Rounds:{nrounds}, Periods{nperiods}, Tokens:{ntokens}, Buyers:{nbuyers}, Sellers{nsellers}, R1 to R4:{R1, R2, R3, R4}")

# data storage
token_values_data = []
step_data = []
period_data = []
round_data = []
buyer_strategies = ['Truthteller']*nbuyers
seller_strategies = ['Truthteller']*nsellers
period_profits_buyers = []
period_profits_sellers = []

# Begin game
for rnd in range(nrounds):
    
    if verbose == 1:
        print('\nRound:',rnd)
        
    # generate token values
    token_values = np.round(round_setup(ntokens, nbuyers, nsellers, R1, R2, R3, R4),1)
    token_values_data.append([token_values,ntokens, nbuyers, nsellers, R1, R2, R3, R4])

    # data from round
    rounds_profits_buyers = []
    rounds_profits_sellers = []

    # begin first trading period
    for period in range(nperiods):
        
        # initialize strategies
        buyers = [TruthTeller(token_values[i],buyer=True) for i in range(nbuyers)]
        sellers = [TruthTeller(token_values[i],buyer=False) for i in range(nbuyers,nbuyers+nsellers)]

        # compute equilibrium
        p_eqbm, q_eqbm = compute_demand_supply(token_values)

        # period data
        period_bids = []
        period_asks = []
        period_prices = []
        period_sales = []
        tokens_sold = 0

        if verbose == 1:
            print('\n\tPeriod:',period)
            
        # begin bid/ask and buy/sell steps
        for step in range(nsteps):
            
            if verbose == 1:
                print(f'\n\t\tStep:{step}')
                
            # obtain bids and asks
            bids = [i.bid() for i in buyers]
            asks = [i.ask() for i in sellers]
            if verbose == 1:
                print('\t\tbids:',bids)
                print('\t\tasks:',asks)
            bids = np.array(bids)
            asks = np.array(asks)
            #bids[bids == None] = np.nan
            #asks[asks == None] = np.nan
            
            # obtain current bid and ask
            try:
                current_bid_idx = np.nanargmax(bids)
                current_ask_idx = np.nanargmin(asks)
            except:
                break
            current_bid = np.nanmax(bids)
            current_ask = np.nanmin(asks)
            if verbose == 1:
                print('\t\tcurrent bid:',current_bid_idx)
                print('\t\tcurrent ask:',current_ask_idx)
                
            # move to buy/sell step 
            buyer_acceptance = buyers[current_bid_idx].buy(current_bid,current_ask)
            seller_acceptance = sellers[current_ask_idx].sell(current_bid,current_ask)
            price = np.nan
            if buyer_acceptance:
                if seller_acceptance:
                    price = np.random.choice([current_bid,current_ask])
                    buyers[current_bid_idx].transact(price)
                    sellers[current_ask_idx].transact(price)
                    tokens_sold += 1
                    if verbose == 1:
                        print(f'\t\t{current_ask_idx} sells to {current_bid_idx} at {price}!')
            else:
                tokens_sold += 0
                
            # record time-step data
            period_bids.append(bids)
            period_asks.append(asks)
            period_prices.append(price)
            period_sales.append(tokens_sold)
            period_profits_buyers.append([rnd,period,step]+[buyer.period_profit for buyer in buyers])
            period_profits_sellers.append([rnd,period,step]+[seller.period_profit for seller in buyers])
            step_data.append([rnd,period,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,buyer_acceptance,seller_acceptance,price,p_eqbm, q_eqbm,nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4, minprice, maxprice, ntimes, buyer_strategies, seller_strategies])

        # record trading period data
        filtered_prices = [price for price in period_prices if price is not None]
        period_data.append([rnd,period,p_eqbm,q_eqbm,period_prices,token_values,buyers_profits,sellers_profits,buyer_strategies,seller_strategies])
        if verbose == 1:
            print(f'\n\tTotal Trades: {len(filtered_prices)}')
            print(f'\tPrices: {period_prices}')
            # plot the result 
            plot_period_results(period_bids, period_asks, period_prices, period_sales, token_values, ntokens, nbuyers, nsellers, R1, R2, R3, R4)

    # record trading round data
    buyers_profits = [buyer.round_profit for buyer in buyers]
    sellers_profits = [seller.round_profit for seller in buyers]
    round_data.append([rnd,p_eqbm,q_eqbm,buyers_profits,sellers_profits,buyer_strategies,seller_strategies])

--------Trading Game Start--------
Rounds:4, Periods4, Tokens:4, Buyers:4, Sellers4, R1 to R4:(20, 20, 20, 20)


In [171]:
profit_analysis(period_profits_buyers, period_profits_sellers)

Avg Buyer Profits: [ 4.459   10.494    7.1695   8.74575]
Avg Seller Profits: [ 4.459   10.494    7.1695   8.74575]


### Collect Data

In [23]:
import pandas as pd
step_column_names = [
    "rnd", "period", "step", "bids", "asks", "current_bid", "current_bid_idx",
    "current_ask", "current_ask_idx", "buyer_acceptance", "seller_acceptance",
    "price", "p_eqbm", "q_eqbm", "nrounds", "nperiods", "ntokens", "nbuyers",
    "nsellers", "R1", "R2", "R3", "R4", "minprice", "maxprice", "ntimes",
    "buyer_strategies", "seller_strategies"
]
period_column_names = [
    "rnd", "period", "p_eqbm", "q_eqbm", "prices", "token_values", "buyers_profits",
    "sellers_profits", "buyer_strategies", "seller_strategies"
]
round_column_names = [
    "rnd", "p_eqbm", "q_eqbm", "buyers_profits", "sellers_profits",
    "buyer_strategies", "seller_strategies"
]

#data.append([rnd,period,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,buyer_acceptance,seller_acceptance,price,nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4, minprice, maxprice, ntimes])
df_step = pd.DataFrame(step_data, columns = step_column_names)
df_period = pd.DataFrame(period_data, columns = period_column_names)
df_round = pd.DataFrame(round_data, columns = round_column_names)


In [24]:
df_round.head()

,rnd,p_eqbm,q_eqbm,buyers_profits,sellers_profits,buyer_strategies,seller_strategies
0,0,[],None,"[0, 0, 0, 0]","[0, 0, 0, 0]","[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."


In [25]:
df_period.head()

,rnd,period,p_eqbm,q_eqbm,prices,token_values,buyers_profits,sellers_profits,buyer_strategies,seller_strategies
0,0,0,[],None,"[55.8, 31.7, 32.7, 32.9, 49.9, 33.6, 34.7, 48....","[[40.1, 49.9, 53.4, 42.8, 45.2, 39.2, 45.8, 48...","[39.800000000000004, 51.999999999999986, 4.5, ...","[39.800000000000004, 51.999999999999986, 4.5, ...","[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."


In [26]:
df_step.tail()

,rnd,period,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,buyer_acceptance,...,nsellers,R1,R2,R3,R4,minprice,maxprice,ntimes,buyer_strategies,seller_strategies
20,0,0,20,"[40.1, 41.6, 39.1, 37.4]","[42.9, 43.8, nan, 42.6]",41.6,1,42.6,3,False,...,4,20,20,20,20,1,2000,25,"[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."
21,0,0,21,"[40.1, 41.6, 39.1, 37.4]","[42.9, 43.8, nan, 42.6]",41.6,1,42.6,3,False,...,4,20,20,20,20,1,2000,25,"[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."
22,0,0,22,"[40.1, 41.6, 39.1, 37.4]","[42.9, 43.8, nan, 42.6]",41.6,1,42.6,3,False,...,4,20,20,20,20,1,2000,25,"[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."
23,0,0,23,"[40.1, 41.6, 39.1, 37.4]","[42.9, 43.8, nan, 42.6]",41.6,1,42.6,3,False,...,4,20,20,20,20,1,2000,25,"[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."
24,0,0,24,"[40.1, 41.6, 39.1, 37.4]","[42.9, 43.8, nan, 42.6]",41.6,1,42.6,3,False,...,4,20,20,20,20,1,2000,25,"[Truthteller, Truthteller, Truthteller, Trutht...","[Truthteller, Truthteller, Truthteller, Trutht..."
